# Here we will calculate core microbiomes for each coral genus (field_host_genus_id) within each compartment (BiologicalMatter).

# [Part 1]

In [1]:
cd ..

/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment


In [2]:
#First lets import some libraries and preferences
from os.path import join,abspath
from os import listdir
from os.path import splitext

#And assign links
input_dir = abspath("input/")
otu_table = abspath("input/otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000.biom")
tree_fp = abspath("input/rep_set.tre")
mapping = abspath("input/gcmp16S_map_r20.txt")
output_dir = abspath("output/")
compartment_folder = join(output_dir,'compartments')

In [10]:
mkdir $compartment_folder

mkdir: /Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment/output/compartments: File exists


In [11]:
#We will filter out only Coral Mucus, Coral Skeleton and Coral Tissue using the BiologicalMatter field
!filter_samples_from_otu_table.py -i $otu_table -o $compartment_folder/mucus_only.biom -m $mapping -s 'BiologicalMatter:Coral Mucus'
!filter_samples_from_otu_table.py -i $otu_table -o $compartment_folder/skeleton_only.biom -m $mapping -s 'BiologicalMatter:Coral Skeleton'
!filter_samples_from_otu_table.py -i $otu_table -o $compartment_folder/tissue_only.biom -m $mapping -s 'BiologicalMatter:Coral Tissue'

#For now, we will not filter out Reef Sediment and Reef Water using the sample_type field
#!filter_samples_from_otu_table.py -i $otu_table -o $compartment_folder/sediment_only.biom -m $mapping -s 'sample_type:Reef Sediment'
#!filter_samples_from_otu_table.py -i $otu_table -o $compartment_folder/water_only.biom -m $mapping -s 'sample_type:Reef Water'

In [16]:
# Within each compartment (BiologicalMatter), we will calculate the core microbiome for each coral genus (field_host_genus_id)

# Using compartment specific biom files, we will use compute_core_microbiome.py with valid_states flag to get cores for each genus/compartment combination.

# NOTE: Caulastrea, Dipsastraea, Leptastrea, Lobophytum, Millepora, Mycedium, Palythoa removed due to low sample sizes

fraction = .5

for c in listdir(compartment_folder):
    if not c.endswith('.biom'):
        continue
    compartment_specific_folder = join(compartment_folder,c.rstrip('.biom'))
    for gen in ['Acanthastrea','Acropora','Alveopora','Astrea','Cyphastrea','Diploastrea','Echinopora','Favia','Favites','Fungia','Galaxea','Goniastrea','Hydnophora','Isopora','Lobophyllia','Merulina','Montipora','Pachyseris','Pavona','Physogyra','Platygyra','Pocillopora','Porites','Psammocora','Seriatopora','Stylophora','Symphyllia','Turbinaria']:
        cur_compartment_otu_table = join(compartment_folder,"%s" %c)
        cur_compartment_genus = join(compartment_specific_folder,"%s" %gen)
        print(c)
        print(gen)
        cur_valid_state = ':'.join(['field_host_genus_id',"%s" %gen])
        print(cur_compartment_otu_table)
        !compute_core_microbiome.py -i $cur_compartment_otu_table -o $cur_compartment_genus --min_fraction_for_core $fraction --num_fraction_for_core_steps 6 --mapping_fp $mapping --valid_states $cur_valid_state

mucus_only.biom
Acanthastrea
/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment/output/compartments/mucus_only.biom
Traceback (most recent call last):
  File "/macqiime/anaconda/bin/compute_core_microbiome.py", line 171, in <module>
    main()
  File "/macqiime/anaconda/bin/compute_core_microbiome.py", line 156, in main
    write_biom_table(core_table, output_table_fp)
  File "/macqiime/anaconda/lib/python2.7/site-packages/qiime/util.py", line 569, in write_biom_table
    "Attempting to write an empty BIOM table to disk. "
qiime.util.EmptyBIOMTableError: Attempting to write an empty BIOM table to disk. QIIME doesn't support writing empty BIOM output files.
mucus_only.biom
Acropora
/Users/FJPollock/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiome

In [6]:
# Convert biom (containing all OTUs) to TSV in order to import taxonomy into tree

!biom convert -i $otu_table -o input/otu_table_mc2_wtax_no_pynast_failures_no_organelles_even1000_TSV.txt --to-tsv --header-key taxonomy

# Now, move to R notebook... 3c_core_microbiomes_by_compartment_R_v2.2.ipynb [Part 2]

# [Part 3]

In [3]:
# Create a tree for the core OTUs at each level

for c in listdir(compartment_folder):
    if not c.endswith('_only'):
        continue
    compartment_specific_folder = join(compartment_folder,c)
    for otus_to_keep in listdir(compartment_specific_folder):
        if not otus_to_keep.endswith('_otus_to_keep.txt'):
            continue
        otus_to_keep = join(compartment_specific_folder,otus_to_keep)
        trimmed_tree_out = ".".join([otus_to_keep.rstrip(".txt"),"tre"])
        !filter_tree.py -i $tree_fp -t $otus_to_keep -o $trimmed_tree_out

# Compute core vs. variable microbiome

In [3]:
# Limit biom files to only genus of interest and convert level-specific biom files to TSV to feed into R:

# Define levels of interest
levels = ['50','70','90']

for c in listdir(compartment_folder):
    if not c.endswith('_only'):
        continue
    compartment_specific_folder = join(compartment_folder,c)
    for gen in ['Acanthastrea','Acropora','Alveopora','Astrea','Cyphastrea','Diploastrea','Echinopora','Favia','Favites','Fungia','Galaxea','Goniastrea','Hydnophora','Isopora','Lobophyllia','Merulina','Montipora','Pachyseris','Pavona','Physogyra','Platygyra','Pocillopora','Porites','Psammocora','Seriatopora','Stylophora','Symphyllia','Turbinaria']:
        genus_specific_folder = join(compartment_specific_folder,gen)
        for core_level in levels:
            biom_file = ''.join(["core_table_","%s" %core_level,".biom"])
            genus_level_specific_biom = join(genus_specific_folder,biom_file)
            tsv_file = ''.join(["core_table_","%s" %core_level,"_TSV.txt"])
            genus_level_specific_tsv_out = join(genus_specific_folder,tsv_file)
            genus_level_specific_biom_genus_only_file = ''.join(["core_table_","%s" %core_level,"_","%s" %gen,"_only.biom"])
            genus_level_specific_biom_genus_only = join(genus_specific_folder,genus_level_specific_biom_genus_only_file)
            cur_valid_state = ':'.join(['field_host_genus_id',"%s" %gen])
            genus_level_only_tsv_file = ''.join(["core_table_","%s" %core_level,"_","%s" %gen,"_only_TSV.txt"])
            genus_level_only_tsv_out = join(genus_specific_folder,tsv_file)
            print(genus_level_specific_biom_genus_only)
            !filter_samples_from_otu_table.py -i $genus_level_specific_biom -o $genus_level_specific_biom_genus_only -m $mapping -s $cur_valid_state
            !biom convert -i $genus_level_specific_biom_genus_only -o $genus_level_only_tsv_out --to-tsv --header-key taxonomy

/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment/output/compartments/mucus_only/Acanthastrea/core_table_50_Acanthastrea_only.biom
/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment/output/compartments/mucus_only/Acanthastrea/core_table_70_Acanthastrea_only.biom
Error in filter_samples_from_otu_table.py: option -i: file does not exist: '/Volumes/Macintosh_HDD/Dropbox/Coral_Microbiomes_Postdoc/GCMP_OSU_PSU_Shared_Folder/Coral_microbe_coevolution/Projects/Australia_Coevolution_Paper/16S_analysis/3c_core_microbiomes_by_compartment/output/compartments/mucus_only/Acanthastrea/core_table_70.biom'

If you need help with QIIME, see:
http://help.qiime.org
Usage: biom convert [options] {-i/--input-fp INPUT-FP -o

# Now, move to R notebook... 3c_core_microbiomes_by_compartment_R_v2.2.ipynb [Part 4]